# Applying Welch's t-test on ASCAD

In [6]:
from tqdm import tqdm
from src.data.slices import make_slice
from src.tools.traceloader import ASCAD
import numpy as np
from src.tvla.main import tvla
from src.tvla.t import make_t_test

In [7]:
ascad = ASCAD()

In [21]:
ascad.masked.profile.filter_traces(0).shape, ascad.default.profile.filter_traces(0).shape

((753, 1400), (760, 1400))

In [45]:
P_VALUE = 5
PT_OF_INTEREST = (50, 80)

def prepare(make_test, samples, p=P_VALUE, poi=PT_OF_INTEREST):
    sample_sizes = [len(samples.filter_traces(i)) for i in range(ascad.key_size)]
    mss = min(sample_sizes)

    print(mss)

    slc = np.array([make_slice(samples.filter_traces(i), mss) for i in range(ascad.key_size)])

    print(slc.shape)
    slc_mv = np.moveaxis(np.array((slc.mean(axis=2, dtype=np.float128), slc.var(axis=2, dtype=np.float128))), 0, -1)

    test = make_test(mss)

    return test, slc_mv

# masked = prepare(make_t_test, ascad.masked.profile)

default = prepare(make_t_test, ascad.default.profile)
random = prepare(make_t_test, ascad.random.profile)

704
(256, 4, 176, 1400)
704
(256, 4, 176, 1400)


In [62]:
tvla(test=default[0], left=default[1][0], right=default[1][208], p=1)

(False, False)

In [47]:
tvla(test=random[0], left=random[1][0], right=random[1][208], p=1)

(False, False)

In [ ]:
def key_tvla(test, slices: np.array, key_size=ascad.key_size):
    res = np.array(np.zeros((key_size, key_size)), dtype=int)
    progress = tqdm(total=key_size)
    ff, tf, ft, tt = 0, 0, 0, 0

    for i in range(key_size):
        a = slices[i]
        for j in range(key_size):
            b = slices[j]

            if i != j:
                la = tvla(test=test, left=a, right=b, p=.00001)
                enc = 2 * la[0] + la[1]
                res[i][j] = enc

                ff += enc == 0
                ft += enc == 1
                tf += enc == 2
                tt += enc == 3

        progress.set_description(f"Total ({[ff, ft, tf, tt]})")
        progress.update()

    progress.close()
    return res


res_default = key_tvla(*default)
res_random = key_tvla(*random)

In [10]:
res_default.sum()

186178